In [ ]:
# Instalamos las dependencias necesarias
#!pip install ultralytics roboflow
#!pip install wandb  # Para seguimiento de entrenamiento

from ultralytics import YOLO
from roboflow import Roboflow
import yaml
import os



from roboflow import Roboflow
rf = Roboflow(api_key="TRhCHcFYG8NiMg7s28p2")
project = rf.workspace("vittaprueba").project("logo_model")
version = project.version(1)
dataset = version.download("yolov8")
                

# Configuración personalizada para logos
def crear_config_personalizada(ruta_yaml_original):
    with open(ruta_yaml_original, 'r') as file:
        config = yaml.safe_load(file)
    
    # Añadimos configuración específica para logos
    config['mosaic'] = 0.75  # Aumenta la variabilidad del dataset
    config['mixup'] = 0.1    # Ayuda con logos parcialmente visibles
    config['copy_paste'] = 0.3  # Útil para logos
    config['degrees'] = 15   # Rotación limitada para logos
    config['scale'] = 0.5    # Escala para manejar logos de diferentes tamaños
    
    # Guardamos la configuración personalizada
    ruta_nueva = 'config_logos.yaml'
    with open(ruta_nueva, 'w') as file:
        yaml.dump(config, file)
    return ruta_nueva

# Inicializar modelo
modelo = YOLO('yolov8n.pt')  # Comenzamos con un modelo pre-entrenado

# Configurar hiperparámetros optimizados para detección de logos
hiperparametros = {
    'epochs': 150,          # Más épocas para mejor precisión
    'batch': 16,           # Ajusta según tu GPU
    'imgsz': 640,          # Tamaño óptimo para logos
    'patience': 30,        # Early stopping
    'optimizer': 'Adam',   # Optimizador adaptativo
    'lr0': 0.001,         # Learning rate inicial
    'lrf': 0.01,          # Learning rate final
    'momentum': 0.937,     # Momentum para el optimizador
    'weight_decay': 0.0005,# Regularización
    'warmup_epochs': 3,    # Calentamiento gradual
    'cos_lr': True,       # Learning rate cíclico
    'close_mosaic': 10,   # Desactiva mosaico en últimas épocas
    'label_smoothing': 0.1,# Suavizado de etiquetas
    'device': '0',        # GPU
    'augment': True,      # Aumentación de datos
}

# Crear y usar configuración personalizada
config_path = crear_config_personalizada(f"{dataset.location}/data.yaml")

# Entrenar el modelo
resultados = modelo.train(
    data=config_path,
    **hiperparametros
)

# Evaluar el modelo
metricas = modelo.val()

# Exportar el modelo en diferentes formatos
modelo.export(format='onnx')  # ONNX para despliegue
modelo.export(format='torchscript')  # TorchScript para producción

# Función para probar el modelo
def probar_modelo(modelo, imagen_path):
    resultados = modelo.predict(imagen_path, conf=0.25)
    for r in resultados:
        print(f"Logos detectados: {r.boxes.cls}")
        print(f"Confianza: {r.boxes.conf}")
    return resultados

# Guardar las mejores métricas
with open('metricas_modelo.txt', 'w') as f:
    f.write(f"mAP: {metricas.box.map}\n")
    f.write(f"Precisión: {metricas.box.p}\n")
    f.write(f"Recall: {metricas.box.r}\n")